#Install packages and load libraries

Install non-native packages



In [1]:
%%capture
!pip install scikit-learn==0.22.2.post1
!pip install ipynb

Import common packages

In [2]:
import cProfile, datetime, glob
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle, pstats, shutil, sys
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.manifold import TSNE
from sklearn.metrics import accuracy_score, auc, average_precision_score
from sklearn.metrics import plot_precision_recall_curve, precision_recall_curve
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import f1_score
from sklearn.preprocessing import MinMaxScaler

Enable Google Colab, mount drives, and load proprietary modules...

In [3]:
%%capture
from google.colab import drive, files
drive.mount('/content/drive', force_remount=False)
sys.path.insert(0, '/content/drive/My Drive/Colab Notebooks/Primary Research/JLMwSCBC_notebook/code/code_toPublish')   
import ppiPrediction_v2
import dataProcessing
import modelTraining
import modelEvaluating                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fexperimentsandconfigs%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:
4/1AX4XfWhTHeMQYsBQM4l6cRqCxg6OS5jAQENwnKs7PpyDufhrAZIdJ29uwoo


#Define file paths and key variables

In [4]:
global dateSignature
dateSignature = datetime.datetime.now().strftime("%Y-%m-%d")

#directories
workDir = '/content/drive/My Drive/Colab Notebooks/Primary Research/JLMwSCBC_notebook/'
LMprotein_complexesDir = workDir + 'proteinComplexes/Lugo-Martinez/'
GWprotein_complexesDir = workDir + 'proteinComplexes/Drew_LM_Wilkins/'

In [5]:
dataPartition_dir = workDir + 'dataPartitions/drew+jlm+kaggle+gtex+phantom/'

origLM_cellSpecific_all7_featuresPath_training = GWprotein_complexesDir + 'integrated_cell-specific_MM+TSCS+Exp+Abun+Loc+Kaggle+gTex+phantom+SCBC_features_training(edit).tsv'
expandedLM_cellSpecific_all7_featuresTraining_dataPath = GWprotein_complexesDir + 'integratedExpanded_cell-specific_MM+TSCS+Exp+Abun+Loc+Kaggle+gTex+phantom+SCBC_features_training(edit).tsv'
origLM_cellSpecific_all7_featuresPath_trainingLabels = GWprotein_complexesDir + 'integrated_cell-specific_MM+TSCS+Exp+Abun+Loc+Kaggle+SCBC_labels_training_all_model.tsv'
trainingLabels_path = dataPartition_dir + 'integrated_MM+TSCS+Exp+Abun+Loc+Kaggle+gTex+phantom+SCBC_features_trainingLabels(edit).tsv'

origLM_cellSpecific_all7_featuresPath_test = GWprotein_complexesDir + 'integrated_MM+TSCS+Exp+Abun+Loc+Kaggle+gTex+phantom+SCBC_features_test(edit).tsv'
expandedLM_cellSpecific_all7_featurestest_dataPath = GWprotein_complexesDir + 'integratedExpanded_MM+TSCS+Exp+Abun+Loc+Kaggle+gTex+phantom+SCBC_features_test(edit).tsv'
origLM_cellSpecific_all7_featuresPath_testLabels = GWprotein_complexesDir + 'integrated_cell-specific_MM+TSCS+Exp+Abun+Loc+Kaggle+SCBC_labels_test_all_model.tsv'
testLabels_path = dataPartition_dir + 'integrated_MM+TSCS+Exp+Abun+Loc+Kaggle+gTex+phantom+SCBC_features_testLabels(edit).tsv'

STRING_protLinks_detailedChannel_path = workDir + 'proteinComplexes/9606.protein.links.detailed.v11.0.txt'
STRING_protLinks_aliasesPath = workDir + 'proteinComplexes/9606.protein.aliases.v11.0.txt'

In [6]:
fold = 1
prtnCount = 0
for prtn in np.arange(4249):
  if len(glob.glob(workDir + 'models/models_integrated_cell-specific_MM+TSCS+Exp+Abun+Loc+Kaggle+SCBC_5CV_{0}/model_integrated_cell-specific_MM+TSCS+Exp+Abun+Loc+Kaggle+SCBC_5CV_{0}_Partition_{1}_*.pkl'.format(fold, prtn))) > 0:
    prtnCount+=1
print('The number of models built for CV fold {0} is {1}'.format(fold, prtnCount))

The number of models built for CV fold 1 is 4066


In [7]:
fold = 2
prtnCount = 0
for prtn in np.arange(4249):
  if len(glob.glob(workDir + 'models/models_integrated_cell-specific_MM+TSCS+Exp+Abun+Loc+Kaggle+SCBC_5CV_{0}/model_integrated_cell-specific_MM+TSCS+Exp+Abun+Loc+Kaggle+SCBC_5CV_{0}_Partition_{1}_*.pkl'.format(fold, prtn))) > 0:
    prtnCount+=1
print('The number of models built for CV fold {0} is {1}'.format(fold, prtnCount))

The number of models built for CV fold 2 is 3805


In [8]:
fold = 3
prtnCount = 0
for prtn in np.arange(4249):
  if len(glob.glob(workDir + 'models/models_integrated_cell-specific_MM+TSCS+Exp+Abun+Loc+Kaggle+SCBC_5CV_{0}/model_integrated_cell-specific_MM+TSCS+Exp+Abun+Loc+Kaggle+SCBC_5CV_{0}_Partition_{1}_*.pkl'.format(fold, prtn))) > 0:
    prtnCount+=1
print('The number of models built for CV fold {0} is {1}'.format(fold, prtnCount))

The number of models built for CV fold 3 is 3553


In [ ]:
fold = 0
prtnCount=0
for prtn in np.arange(4249):
  if not len(glob.glob(workDir + 'models/models_integrated_cell-specific_MM+TSCS+Exp+Abun+Loc+Kaggle+SCBC_5CV_{0}/model_integrated_cell-specific_MM+TSCS+Exp+Abun+Loc+Kaggle+SCBC_5CV_{0}_Partition_{1}_*.pkl'.format(fold, prtn))) > 0:
    print('Need for 5CV fold {0}--partition {1}'.format(fold, prtn))
    prtnCount+=1

print('The total number of missing partitions is {0}'.format(prtnCount))

In [13]:
for fold in np.arange(5):
  print('for fold {0}'.format(fold))
  prtnCount=0
  for prtn in np.arange(4249):
    if not len(glob.glob(workDir + 'models/models_integrated_cell-specific_MM+TSCS+Exp+Abun+Loc+Kaggle+SCBC_5CV_{0}/model_integrated_cell-specific_MM+TSCS+Exp+Abun+Loc+Kaggle+SCBC_5CV_{0}_Partition_{1}_*.pkl'.format(fold, prtn))) > 0:
      print('Need for 5CV fold {0}--partition {1}'.format(fold, prtn))
      prtnCount+=1

  print('The total number of missing partitions is {0}'.format(prtnCount))

for fold 0
The total number of missing partitions is 0
for fold 1
The total number of missing partitions is 0
for fold 2
The total number of missing partitions is 0
for fold 3
Need for 5CV fold 3--partition 2627
Need for 5CV fold 3--partition 3266
Need for 5CV fold 3--partition 3280
Need for 5CV fold 3--partition 3282
Need for 5CV fold 3--partition 3283
Need for 5CV fold 3--partition 3287
Need for 5CV fold 3--partition 3288
Need for 5CV fold 3--partition 3289
Need for 5CV fold 3--partition 3290
Need for 5CV fold 3--partition 3291
Need for 5CV fold 3--partition 3293
Need for 5CV fold 3--partition 3294
Need for 5CV fold 3--partition 3295
Need for 5CV fold 3--partition 3296
Need for 5CV fold 3--partition 3302
Need for 5CV fold 3--partition 3304
Need for 5CV fold 3--partition 3305
Need for 5CV fold 3--partition 3307
Need for 5CV fold 3--partition 3308
Need for 5CV fold 3--partition 3310
Need for 5CV fold 3--partition 3312
Need for 5CV fold 3--partition 3313
Need for 5CV fold 3--partition 3

#Load Data

In [ ]:
trainingData = pd.read_csv(origLM_cellSpecific_all7_featuresPath_training, sep='\t')
trainingLabels = pd.read_csv(trainingLabels_path, sep='\t', names=['label', 'IDi', 'IDii'], header=None)

In [ ]:
trainingData.shape

(491524, 281)

In [ ]:
trainingData.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.000731,NaN,0.562487,0.895858,0.330261,-0.020734,-0.138685,1.0,0.5,0.625000,0.5,NaN,NaN,NaN,NaN,NaN,0.716575,0.854239,0.975442,0.840369,0.802764
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.000336,-0.001248,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.000812,NaN,0.644494,0.026106,0.603865,0.830032,1.0,1.0,1.000000,0.0,NaN,NaN,NaN,NaN,NaN,0.808222,0.580970,0.964623,0.845938,0.861357
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.462464,1.0,NaN,-0.003727,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.003470,0.872041,0.655694,0.070300,0.437760,0.103568,1.0,0.0,0.333333,1.0,NaN,NaN,NaN,NaN,NaN,-0.154853,-0.298179,-0.111857,-0.084085,-0.240541
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.319827,1.0,NaN,0.101370,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.450847,0.884051,-0.080513,-0.067202,0.128490,0.0,0.0,0.000000,1.0,NaN,NaN,NaN,NaN,NaN,-0.013903,-0.301384,0.236531,0.273680,0.308547
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.000371,NaN,0.912048,0.814470,0.399663,0.095235,0.308318,0.0,0.0,0.000000,1.0,NaN,NaN,NaN,NaN,NaN,0.690803,0.954442,0.803877,0.853439,0.801518


In [ ]:
trainingLabels.shape

(491524, 3)

In [ ]:
trainingLabels.head()

,label,IDi,IDii
0,1,6627,51428
1,1,23013,10594
2,1,11338,4869
3,1,10250,11157
4,1,25804,6633


#Extract class probabilities for training data

In [ ]:
expName = 'cellSpecific_integratedPairs_Fantom+GTEx+Hein+HPA+Huttlin+Lugo-Martinez+Kaggle+NCI-60+SCBC+Wan'
partitionsFile = workDir + 'models/models_{0}/savedPartitions.pkl'.format(expName)
partitionsSorted = pickle.load(open(partitionsFile, 'rb'))
trainedModels_properties = pd.read_csv(workDir + f'models/models_{expName}/Partition Specific '
                                                  f'{expName} Training Performance_2021-06-15.csv')

In [ ]:
candidatePartitions, candidatePartitions_preds, candidatePartitions_predsProbs, modelsEval_properties = \
    modelEvaluating.evalPartitions(partitionsSorted, trainedModels_properties, 
                                   trainingData, trainingLabels, workDir, expName, suffix='_trainingData')

Evaluating models by training data partitions on testing data
____________________________________________________________
